## This notebook should not be relaunched once you have the df_file_final.csv up

The goal of this notebook is to create a **DataFrame** containing all phrases link to the company in each document, but also to add some features (when available) such as Title, Frequency and Count of company name

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_pandas
from tqdm import tqdm_notebook as tqdm
import extract_text
import text_summarization

In [4]:
df_files = pd.read_csv("submission_mapper.csv",delimiter="|")
df_files['file_path'] = df_files.apply(lambda x : "files/"+x.company+"/"+x.filename,axis=1)
df_files['sentiment'] = np.nan
df_files.head()

,id,filename,company,file_path,sentiment
0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,NaN
1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,NaN
2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,NaN
3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,NaN
4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,NaN


In [5]:
keppelWord = ['keppel corporation', 'keppel capital', 'keppel reit', 'keppel infrastructure','keppel o&m', 'keppel energy', 'keppel offshore', 'keppel dhcs', 'keppel fairway', 'keppel professorship', 'keppel reit', 'keppel group', 'keppel land' , 'keppel singmarine', 'keppel fels', 'keppel shipyard','keppel shipyard' , 'keppel gas','keppel t&t', 'keppel bay', 'keppel telecom','keppel corp','keppel seghers','keppel center','keppel cebu','keppel thai','keppel philippines','keppel tatle','keppel  telecoms','keppel tower']
prudentialWord = ['prudential mutual','prudential plc']
phrases = pd.DataFrame(data=[],columns=["Sentences"])

,Sentences


In [8]:
##Code to drop non interesting documents
def doWeDrop(sentences):
    """Return true for document not talking about either Keppel or Prudential"""
    isValid = False
    for phrase in sentences:
        lowered = phrase.lower()
        for expr1 in keppelWord:
            if(expr1 in lowered):
                isValid = True
        for expr2 in prudentialWord:
            if(expr2 in lowered):
                isValid = True
    return not isValid

In [9]:
##Fill sentences, meaning extract text and summarize it.
def fillSentences(path,company):  
    try:
        text = extract_text.apply(path)
        if doWeDrop(text):
            return ["-500"]
        important_sents = text_summarization.apply(text,company)
        print(len(important_sents))
        return important_sents
    except:
        print("error")
        return [" "]

In [ ]:
##Dont't relaunch this it is used to fill the DF, it is realllllly long
tqdm().pandas(desc="Progress:")
df_files['sentences'] = ""
phrases["Sentences"] = df_files.progress_apply(lambda row : fillSentences(row["file_path"],row["company"].lower()),axis=1)

In [9]:
##Load sentences into df_files. (Apply was returning a error due to dimension problem so we used .loc)
df_files_new = df_files
for i in range(len(phrases.Sentences)):
    df_files_new.loc[i,'sentences'] = phrases.Sentences[i]


In [11]:
## Save in a csv.
df_files_new.to_csv("safe/df_file_final.csv")
df_files_new.head()

,Unnamed: 0,id,filename,company,file_path,sentiment,sentences
0,0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,NaN,([('Keppel Capital aims to create value and de...
1,1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,NaN,([('Company Backg round Keppel is a diversifie...
2,2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,NaN,([('The new SOCAR-Keppel Shipyard will be deve...
3,3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,NaN,['-500']
4,4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,NaN,([('Jurong Shipyard Pte Ltd (Songa Eclipse) Ju...
